## This notebook is used to extract num_accident feature for train and test dataset

In [1]:
import pandas as pd
from datetime import datetime

### First build a fuction to extract num_feature

In [2]:
#If the data is taken from csv 
#If data is from bigquery, delect the process datetime
def get_accident_count(dat_1, dat_2):
    """
        @param dat_1: [start_lati, start_long, end_lati, end_long, 30min before start_time, start_time]
        @param dat_2: # [timestamp, latitude, longitude]
        
    """
    list1 = dat_1.values.tolist() # convert dat_1 to list 
    list2 = dat_2.values.tolist() # convert dat_2 to list
    
    # Process datetime
    for l in list1:
        l[4] = datetime.strptime(l[4].split("+")[0], '%Y-%m-%d %H:%M:%S')
        l[5] = datetime.strptime(l[5].split("+")[0], '%Y-%m-%d %H:%M:%S')
    
    for l in list2:
        l[0] = datetime.strptime(l[0].split("+")[0], '%Y-%m-%d %H:%M:%S')
        
    
    result = [] # keep final result
    wrong_counter = 0 # count exceptions
    
    for i in range(len(list1)):
        l1 = list1[i] # current list
        
        # Convert timestamp to naive timezone
        ts1 = l1[4] # timestamp 1
        ts2 = l1[5] # timestamp 2
        l1[4] = pd.Timestamp(ts1.year, ts1.month, ts1.day, ts1.hour, ts1.minute, ts1.second)
        l1[5] = pd.Timestamp(ts2.year, ts2.month, ts2.day, ts2.hour, ts2.minute, ts2.second)
        try:
            if (l1[0] >= l1[2]):
                start_lat = l1[2]
                end_lat = l1[0]
            else:
                start_lat = l1[0]
                end_lat = l1[2]
            if (l1[1] >= l1[3]):
                start_long = l1[3]
                end_long = l1[1]
            else:
                start_long = l1[1]
                end_long = l1[3]
            start_time = l1[4]
            end_time = l1[5]
            counter = 0
            # For each list in dat_1, check all lists in dat_2
            for j in range(len(list2)):
                l2 = list2[j]
                if (start_lat <= l2[1] and end_lat >= l2[1] and start_long <= l2[2] and end_long >= l2[2] and start_time <= l2[0] and end_time >= l2[0]):
                    counter += 1
            result.append(counter)
        except:
            result.append(-1)
            print("An exception occured during execution!!!!!")
            wrong_counter += 1
            continue
    
    return result

### Extract num_accident for training dataset

In [3]:
dat_1 = pd.read_csv("C:/Users/weiyao/Desktop/train_dataset_1.csv")
dat_2 = pd.read_csv("C:/Users/weiyao/Desktop/train_dataset_2.csv")
dat_1_obj = dat_1[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude','30_minute_before_pickup', 'pickup_datetime' ]]
dat_2_obj = dat_2[['timestamp','latitude','longitude']]

In [ ]:
import time
start = time.time()
count_acc = get_accident_count(dat_1_obj, dat_2_obj)
end = time.time()
print("time elapsed {}".format(end - start))

In [23]:
new_col = pd.Series(count_acc)
dat_1['num_accident'] = new_col.values

In [35]:
dat_1.to_csv('C:/Users/weiyao/Desktop/train_dataset.csv')

### Extract num_accident for testing dataset

In [3]:
dat_1_test = pd.read_csv("C:/Users/weiyao/Desktop/test_dataset_1.csv")
dat_2_test = pd.read_csv("C:/Users/weiyao/Desktop/test_dataset_2.csv")
dat_1_test_obj = dat_1_test[['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude','30_minute_before_pickup', 'pickup_datetime' ]]
dat_2_test_obj = dat_2_test[['timestamp','latitude','longitude']]

In [6]:
import time
start = time.time()
count_acc_test = get_accident_count(dat_1_test_obj, dat_2_test_obj)
end = time.time()
print("time elapsed {}".format(end - start))

time elapsed 22870.629986763


In [7]:
new_col_test = pd.Series(count_acc_test)
dat_1_test['num_accident'] = new_col_test.values

In [9]:
dat_1_test.to_csv('C:/Users/weiyao/Desktop/test_dataset_1111111111.csv')